In [12]:
import wilds
print(wilds.__version__)
from wilds.datasets.camelyon17_dataset import Camelyon17Dataset
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.models import resnet50, ResNet50_Weights

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

torch.backends.cudnn.benchmark = True
from tqdm.auto import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import wandb


2.0.0


In [13]:
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="opent03-team",
    # Set the wandb project where this run will be logged.
    project="wilds_dpddm",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.001,
        "architecture": "resnet50",
        "dataset": "Camelyon17",
        "epochs": 50,
    },
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /h/300/viet/.netrc
wandb: Currently logged in as: opent03 (opent03-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [14]:
dataset = Camelyon17Dataset(root_dir='/h/300/viet/bayesian_dpddm/data/', download=False)

In [15]:
splits = {
    'train': 'train',
    'valid': 'val',
    'dpddm_train': 'id_val',
    'dpddm_id': 'val',
    'dpddm_ood': 'test'
}
dataset_dict = {}

In [16]:
for split in splits:
    dataset_dict[split] = dataset.get_subset(split=splits[split], transform=Compose([Resize((224, 224)), ToTensor()]))

In [17]:
BATCH_SIZE=512
EPOCHS=50
LEARNING_RATE=1e-3

In [18]:
trainloader = torch.utils.data.DataLoader(dataset_dict['train'], batch_size=BATCH_SIZE, num_workers=10, pin_memory=True)
valloader = torch.utils.data.DataLoader(dataset_dict['valid'], batch_size=BATCH_SIZE, num_workers=10, pin_memory=True)

In [19]:
model = resnet50()
model.fc = nn.Linear(in_features=2048, out_features=1)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

#model = nn.DataParallel(model)
model = model.to(device)

In [20]:
def train(model, loader, optimizer, criterion):
    model.train()
    losses = 0
    for images, labels, _ in tqdm(loader, leave=False):
        optimizer.zero_grad()
        images, labels = images.to(device), labels.unsqueeze(1).float().to(device)
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        losses += loss.item() * len(labels)
        optimizer.step()
        wandb.log({
            'inst_train_loss': loss.item()
        })
    return losses
        
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels, _ in tqdm(loader, leave=False):
            images, labels = images.to(device), labels.unsqueeze(1).to(device)
            output = model(images)
            preds = output > 0.5
            correct += torch.sum(preds == labels).item()
            total += len(labels)
    return correct / total

In [ ]:
train_losses = []
train_accs = []
valid_accs = []

for e in range(EPOCHS):
    loss = train(model, trainloader, optimizer, criterion)
    train_acc = evaluate(model, trainloader)
    valid_acc = evaluate(model, valloader)
    
    train_losses.append(loss)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    wandb.log({
        'train_loss': loss,
        'train_acc': train_acc,
        'valid_acc': valid_acc
    })

KeyboardInterrupt: 

In [8]:
!nvidia-smi

Thu Apr 10 23:25:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:3B:00.0 Off |                  Off |
| 30%   19C    P8             17W /  230W |       4MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----